## HuggingFace
https://huggingface.co/
- Hugging Face is a company that specializes in Natural Language Processing (NLP) and provides a comprehensive set of tools and libraries for working with state-of-the-art models.
- One of their most popular contributions is the transformers library, which includes pre-trained models and utilities for training and fine-tuning models like BERT. The library simplifies the process of working with transformer-based models, making them accessible to a broader audience.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# google drive에 임의의 폴더를 만들어서 진행합니다. -> 실습
%cd ./drive/MyDrive/LLM

/content/drive/MyDrive/실습


In [3]:
import torch
import torchvision
print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')


torch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


### Data Preparation : (Korpora)
Korpora는 한글 자연어처리를 위한 데이터셋을 손쉽게 로드해주도록 도와주는 오픈소스 파이썬 라이브러리이다.
- https://github.com/ko-nlp/Korpora

In [4]:


# install 1 : from source
# !git clone https://github.com/ko-nlp/Korpora
# !python setup.py install

# install 2 : using pip
!pip install Korpora==0.2.0rc1


!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

사용 디바이스: cuda:0

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 64.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 31.0MB/s]                           


Korpora의 네이버 영화 댓글 데이터셋으로 진행한다
- https://github.com/e9t/nsmc

In [5]:
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
print('train',train.shape)
print('test',test.shape)

train (150000, 3)
test (50000, 3)


In [9]:
train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


### 데이터 전처리 및 토큰화

문장의 길이를 계산하여 [length] column에 담는다.

In [10]:
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))


In [12]:
train.head(10)

,id,document,label,length
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,19
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,33
2,10265843,너무재밓었다그래서보는것을추천한다,0,17
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,29
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,61
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0,45
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0,21
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0,86
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,22
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1,45


문장의 길이가 단어 기준으로 5이상인 문장만 가져온다.

- loc : location의 약자로 데이터프레임의 행 또는 칼럼의 label이나 boolean array로 인덱싱하는 방법.
    - 칼럼명을 직접 적거나 특정 조건식을 써주어 사람이 읽을 수 있게 데이터에 접근하는 방법
    

In [13]:
train = train.loc[train['length'] > 5]

# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
train.head()

,id,document,label,length
39064,7991845,군대에서 봤는데 1화부터 정주행 시작하게 만든 드라마.... 6화 남기고 따라잡고 ...,1,140
90214,10040334,고사 1에서 범인에게 죽었던 배우가 범인이 되어 돌아옴.,0,31
133461,8281198,장난감으로 전쟁무기를 만든다는내용이 현대사회의 살상무인로봇을 예견하고 만든것같아 섬...,1,57
84182,8073598,스토리 구성도 괜찮고 영화도 잘찍은 한국영화인데 평점이 너무 낮은거아님??? 9점영...,1,65
71435,9219448,우연히봤는데넘잼나네요 ㅎㅎ 남자주인공목소리짱짱 기황후때부터완전달달~~ 근데애기엄마가...,1,66


In [14]:
test = test.loc[test['length'] > 5]
test = test.sample(1000)
test.head()

,id,document,label,length
30205,7459697,쓰레기 더럽게 재미없다..,0,14
13749,8735522,엔딩이 기억에 남는다 평점이 낮은듯하여 별하나 더 단다,1,30
35,1018831,"101, 102.. 103은 못나올것 같다.",0,24
15541,4409970,정말 재밌는듯!,1,8
2679,9321513,이런 어거지 설정에다 주인공 남녀 둘 발연기 쩐다. 마지막 반전이 주는 교훈이 무색...,0,50


In [15]:
MODEL_NAME = 'kykim/bert-kor-base'
# MODEL_NAME='kykim/albert-kor-base'

In [16]:
import torch
from transformers import BertTokenizerFast, AlbertModel
from torch.utils.data import Dataset, DataLoader


class NaverReviewDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성

        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): # 클래스의 인덱스에 접근할 때 자동으로 호출되는 메서드
        # iloc : 행번호로 선택하는 방법
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)       # 2D -> 1D : 1,512 -> 512
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [17]:
# 토크나이저 지정
tokenizer_pretrained = MODEL_NAME

# train, test 데이터셋 생성
train_data = NaverReviewDataset(train, tokenizer_pretrained)
test_data = NaverReviewDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False, num_workers=8)
# train_loader과 test_loader는 iterator이다.
# iterator란 반복을 이용해 어떤 처리를 수행하는 객체를 말한다.
print('train_loader',train_loader)
print('test_loader',test_loader)

train_loader <torch.utils.data.dataloader.DataLoader object at 0x7d6b9747b490>
test_loader <torch.utils.data.dataloader.DataLoader object at 0x7d6b8cdb73a0>


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


생성된 train_loader로부터 1개의 batch만 꺼내서 값을 확인

In [18]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}


- input_ids : 토큰
- attention_mask : 실제 단어가 존재하면 1, 패딩이면 0
- token_type_ids : 문장을 구분하는 id. 단일 문장인 경우에는 전부 0

In [19]:
# 생성된 inputs의 key값 출력
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [20]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

In [21]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [22]:
# label 출력
print(labels)

tensor([0, 0, 1, 1, 0, 1, 0, 1])


### Pretrained Model 호출

In [23]:
from transformers import BertModel
from transformers import AlbertModel

# 모델 생성
model = BertModel.from_pretrained(MODEL_NAME).to(device)
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

결과를 output 에 담은 후 key를 출력해 보면 'last_hidden_state' 와 'pooler_output'이렇게 2개가 출력된다.다.

- last_hidden_state는 배치의 각 시퀀스에서 각 토큰에 대한 숨겨진 표현을 포함한다. 따라서 크기는 (batch_size, seq_len, hidden_size)입니다.

- pooler_output은 배치의 각 시퀀스의 “표현”을 포함하며 크기(batch_size, hidden_size)입니다. 기본적으로 하는 일은 배치(hidden_size 크기의 벡터)에서 각 시퀀스의 [CLS] 토큰의 숨겨진 표현을 가져온 다음 BertPooler를 통해 실행하는 것입니다. 이것은 선형 레이어와 Tanh 활성화 함수로 구성됩니다.

In [24]:
output = model(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [25]:
print(output['last_hidden_state'].shape)
print(output['pooler_output'].shape)


torch.Size([8, 512, 768])
torch.Size([8, 768])


In [26]:
print(output['last_hidden_state'][:,0,:])


tensor([[ 0.2926, -0.3248,  0.2659,  ..., -0.0742, -0.4780,  0.3885],
        [ 0.4464, -0.1872,  0.6020,  ...,  0.0545, -0.4779,  0.5408],
        [-2.1677, -0.9836, -0.5510,  ..., -0.5291, -0.8370, -0.5792],
        ...,
        [ 0.4106,  0.1202,  0.4509,  ...,  0.4587, -0.8346,  0.7352],
        [ 0.5539, -0.0649,  0.2421,  ..., -0.0849, -1.2561,  0.6370],
        [-0.6978,  0.6750, -0.0487,  ..., -1.2351, -0.8963,  0.0999]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [27]:
print(output['pooler_output'])

tensor([[-0.8815,  0.4257, -0.9078,  ...,  0.0422, -0.1953,  0.6971],
        [-0.8427,  0.4335, -0.7918,  ...,  0.8716,  0.0607,  0.6371],
        [ 0.5967,  0.1638, -0.9027,  ..., -0.3862,  0.6422, -0.4995],
        ...,
        [-0.8711,  0.4038, -0.8682,  ...,  0.3048,  0.3644,  0.3480],
        [-0.6790,  0.7396, -0.9539,  ...,  0.0763,  0.4840,  0.8659],
        [-0.6333,  0.7559, -0.9989,  ...,  0.3955,  0.8927,  0.5803]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [28]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(output['last_hidden_state'][:, 0, :])
print(fc_output.shape)
print(fc_output)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([[-0.0302, -0.5379],
        [-0.1904, -0.6359],
        [ 0.1097, -0.2958],
        [ 0.0457, -0.3866],
        [ 0.4618, -0.7528],
        [-0.4076, -0.4738],
        [-0.2375, -0.3293],
        [ 0.1295, -0.9238]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


### Let's fine-tuning

In [29]:
class FineTuningBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super().__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)

        # dropout 설정
        self.dropout = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dropout(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [30]:
BERT_model = FineTuningBertModel(MODEL_NAME)
BERT_model.to(device)

FineTuningBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

Loss & Optimizer

In [31]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(BERT_model.parameters(), lr=1e-5)

In [32]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        # print('output',output.shape) # (8, 2)
        _, index = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += index.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. labels.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 labels.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        # 배치당 training loss,

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [33]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs) # (8,2)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, index = output.max(dim=1)

            # idx.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(index.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

Training Loop

In [34]:
# 최대 Epoch을 지정합니다.
num_epochs = 10

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(BERT_model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(BERT_model, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(BERT_model.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

  0%|          | 0/125 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
training loss: 5.52544, training accuracy: 0.61400: 100%|██████████| 125/125 [01:25<00:00,  1.46batch/s]


[INFO] val_loss has been improved from inf to 0.39890. Saving Model!
epoch 01, loss: 0.69068, acc: 0.61400, val_loss: 0.39890, val_accuracy: 0.83200


training loss: 3.14927, training accuracy: 0.85100: 100%|██████████| 125/125 [01:29<00:00,  1.39batch/s]


[INFO] val_loss has been improved from 0.39890 to 0.36064. Saving Model!
epoch 02, loss: 0.39366, acc: 0.85100, val_loss: 0.36064, val_accuracy: 0.85200


training loss: 2.15012, training accuracy: 0.90200: 100%|██████████| 125/125 [01:29<00:00,  1.39batch/s]


...

epoch 08, loss: 0.01959, acc: 0.99500, val_loss: 0.65845, val_accuracy: 0.86500


training loss: 0.28018, training accuracy: 0.99000: 100%|██████████| 125/125 [01:30<00:00,  1.39batch/s]


epoch 09, loss: 0.03502, acc: 0.99000, val_loss: 0.73953, val_accuracy: 0.84500


training loss: 0.37462, training accuracy: 0.98500: 100%|██████████| 125/125 [01:30<00:00,  1.38batch/s]


epoch 10, loss: 0.04683, acc: 0.98500, val_loss: 0.60997, val_accuracy: 0.85500


In [35]:
# 저장한 state_dict를 로드 합니다.
BERT_model.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

### Inference

In [36]:
class Predictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens) # (1, 2)

        prediction = F.softmax(prediction, dim=1) # (1,2)

        output = prediction.argmax(dim=1).item()

        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [37]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}

# Predictor 인스턴스를 생성합니다.
predictor = Predictor(BERT_model, tokenizer, labels)

In [38]:
# 사용자 입력에 대하여 예측 후 출력을 낼 수 있는 간단한 함수를 생성합니다.
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)


In [39]:
# 부정 리뷰 입력 예시
predict_sentence(predictor)

문장을 입력해 주세요: 난 이 영화가 별로야
[부정 리뷰 입니다.]
확률은: 76.296% 입니다.


### 실습
1. 빈칸을 채우시오.
2. 임의의 5문장의 결과를 스크린샷으로 저장해서 올리시오.

In [ ]:
import pandas as pd

train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')
print('train',train.shape)
print('test',test.shape)
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))


train = train.loc[train['length'] > 5]
train = train.sample(1000)
test = test.loc[test['length'] > 5]
test = test.sample(1000)

In [ ]:
import torch
from transformers import BertTokenizerFast, AlbertModel
from torch.utils.data import Dataset, DataLoader


class NaverReviewDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성

        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): # 클래스의 인덱스에 접근할 때 자동으로 호출되는 메서드
        # iloc : 행번호로 선택하는 방법
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)       # 2D -> 1D : 1,512 -> 512
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [ ]:
MODEL_NAME='kykim/albert-kor-base'
from transformers import AlbertModel
# 토크나이저 지정
tokenizer_pretrained = MODEL_NAME

# train, test 데이터셋 생성
train_data = NaverReviewDataset(train, tokenizer_pretrained)
test_data = NaverReviewDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False, num_workers=8)
# train_loader과 test_loader는 iterator이다.
# iterator란 반복을 이용해 어떤 처리를 수행하는 객체를 말한다.
print('train_loader',train_loader)
print('test_loader',test_loader)

In [ ]:
class FineTuningBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super().__init__()
        # 1. 사전학습 모델 지정
        ###################################################

        ###################################################
        # 2. dropout 설정
        ###################################################

        ###################################################
        # 3. 최종 출력층 정의
        ###################################################

        ###################################################

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 4. 결과의 last_hidden_state 가져옴
        ##############################################

        ##############################################
        # 5.[CLS] 토큰을 가져옴
        ##############################################

        ##############################################
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

albert_model = FineTuningBertModel(MODEL_NAME)
albert_model.to(device)

# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(BERT_model.parameters(), lr=1e-5)

from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 6. 누적 Gradient를 초기화 합니다.
        ##################################################

        ##################################################

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # 7. output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        ################################################################

        ################################################################
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += index.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. labels.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 labels.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")


    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # 8. dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    ################################################################

    ################################################################

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # 9. output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            ################################################################

            ################################################################

            # idx.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(index.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
# 최대 Epoch을 지정합니다.
num_epochs = 10

# 10. checkpoint로 저장할 모델의 이름을 정의 합니다.
###########################################

###########################################

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(albert_model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(albert_model, test_loader, loss_fn, device)

    # 11. val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    ###########################################################################


    ###########################################################################

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

In [ ]:
# 저장한 state_dict를 로드 합니다.
albert_model.load_state_dict(torch.load(f'{model_name}.pth'))

In [ ]:
class Predictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens) # (1, 2)

        prediction = F.softmax(prediction, dim=1) # (1,2)

        output = prediction.argmax(dim=1).item()

        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [ ]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}

# Predictor 인스턴스를 생성합니다.
predictor = Predictor(BERT_model, tokenizer, labels)

# 사용자 입력에 대하여 예측 후 출력을 낼 수 있는 간단한 함수를 생성합니다.
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

In [ ]:
#12. 결과 확인 : 5문장에 대해서 실행하세요
###########################
predict_sentence(predictor)
###########################